# Load and Prepera Dataset

In [1]:
import pandas as pd
import numpy as np

Load dataset, including metadata info on the cavity frequency and length of the run.

In [2]:
def load_dataset(run, path='db/'):
    file = f'{path}AnalyzedDataFFT_Run_{run}_sliced.xlsx'
    meta = pd.read_excel(file,sheet_name=0,header=None)
    freq = pd.read_excel(file,sheet_name=1)             # frequecies
    fft  = pd.read_excel(file,sheet_name=2)             # power
    
    data = pd.DataFrame({'freq':freq[1]})
    
    col = 0
    for col_name in fft.columns: # load all the subruns
        if col > 0:
            data[f'fft{col-1}'] = fft[col_name]
        col += 1
    
    #cavity frequency and number of files in each slice
    center = meta[1][3]
    length = meta[1][8]
    
    print(f'Dataset loaded: {file}\nCavity frequency: {center} Hz\nfft subrun:{len(data.columns)-1}')
    
    return data,center,length

Prepare data to be analyzed:
- Select only a 200 bins window around the cavity frequency
- Rescale data to yottowatt: in general, the average measured power should be known and equal to the noise temperature of the system, so we can rescale the data so that the power at the cavity frequency is $T_{noise} \cdot k_B \cdot \Delta\nu_{bin}$ $[W]$
- Compute weights, i.e. the errors associated to each bin; the error is assumed to be equal to $y_{bin}$. An ulterior term $\frac{1}{\sqrt{N}}$ is added as the bin values are obtained as the average over $N = 2731 \cdot length$ runs

In [3]:
def prep_data(alldata, center, subrun=0, length=500, bin_width=651, nbins=100):
    
    N = length*2731 #N=1365500 if length=500
    
    # select window of 2*nbins bins around center
    # default is to select 200 bins of 651 Hz
    mask = (alldata['freq']>center-bin_width*nbins) & (alldata['freq']<center+bin_width*nbins)
    cavdata = alldata[mask].reset_index(drop=True)    
    
    # scale data to yottowat
    minW = np.min(cavdata[f'fft{subrun}'].copy()) # minimum power in the cavity
    
    # In general, the average measured power should be known and equal to the noise temperature of the system.
    # So we can rescale the data so that the power at the cavity frequency is T_noise k_b B (W)
    ref = minW**(-1) * 3.5*1.38e-23*651/1e-24
    cavdata[f'fft{subrun}'] = ref * cavdata[f'fft{subrun}']  # y' 
 
    freq = cavdata['freq']
    fft = cavdata[f'fft{subrun}']
  
    # set weights
    weights = calc_weights(fft, N)  # -> y'/sqrt(N)
    #weights = np.sqrt(ref)*np.sqrt(cavdata[f'fft{subrun}'])/np.sqrt(N)  #-> sqrt(sigma'/N) = ref*sqrt(y/N)
    
    return freq, fft, weights, ref, N

In [4]:
def calc_weights(data, N=1365500):
    weights = data/np.sqrt(N) # -> y'/sqrt(N)
    return weights

## Load Multiple Run Simultaneously

In [ ]:
def list_files(path):
    listFile=[]
    for root, dirs, files in os.walk(path):
        for file in files:
            listFile.append(os.path.join(root, file))
    return listFile

In [ ]:
def multipleLoad(path='db/', bin_width=651, nbins=100):
    
    # get list of all the files in the directory
    file_list = list_files(path)
    InfoDataset = []
    
    for file in file_list:
        
        # load dataset:
        
        meta = pd.read_excel(file,sheet_name=0,header=None)
        freq = pd.read_excel(file,sheet_name=1)             # frequecies
        fft  = pd.read_excel(file,sheet_name=2)             # power
    
        data = pd.DataFrame({'freq':freq[1]})
    
        col = 0
        for col_name in fft.columns: # load all the subruns
            if col > 0:
                data[f'fft{col-1}'] = fft[col_name]
            col += 1
        #cavity frequency and number of files per run
        center = meta[1][3]
        length = meta[1][8]
        
        
        # preapare dataset:
        
        # select window of 2*nbins bins around center
        # default is to select 200 bins of 651 Hz
        N = length*2731 #N=1365500 if length=500
        
        mask = (data['freq']>center-bin_width*nbins) & (data['freq']<center+bin_width*nbins)
        cavdata = data[mask].reset_index(drop=True)
        
        cavdata["fft"] = cavdata.iloc[:,1:].mean(axis=1)
        cavdata=cavdata[["freq","fft"]]
        
        # scale data to yottowat
        minW = np.min(cavdata["fft"].copy()) # minimum power in the cavity
        # In general, the average measured power should be known and equal to the noise temperature of the system.
        # So we can rescale the data so that the power at the cavity frequency sia T_noise k_b B (W)
        ref = minW**(-1) * 3.5*1.38e-23*651/1e-24 
        cavdata["fft"] = ref * cavdata["fft"]   #y' 
        
        # set weights
        weights = calc_weights(cavdata["fft"], N)  # -> y'/sqrt(N)
        # weights = np.sqrt(ref)*np.sqrt(cavdata[f'fft{subrun}'])/np.sqrt(N)  #-> sqrt(sigma'/N) = ref*sqrt(y/N)       
        
        Info = {"name":file, "length":length, "center":center,
                "cavdata":cavdata, "weights":weights, "ref":ref} 
        InfoDataset.append(Info)
            
    return(InfoDataset)